In [ ]:
# !python3 -m pip install matplotlib

In [ ]:
# !python3 -m pip install pandas

In [ ]:
# !python3 -m pip install mysql-connector

In [ ]:
# !python3 -m pip install pymysql

In [ ]:
# !python3 -m pip install pyyaml

## GET API

In [ ]:
# key = 'GWXLIMM8PJ1W2F2C'
# db_name = 'alphavantage_uat'

In [7]:
## FUNCTION
import pymysql

api_url = 'https://www.alphavantage.co/query'
    
def get_alphavantage(params_dict: dict()) -> dict():
    params = '&'.join([f'{key}={value}' for key, value in params_dict.items()])
    print(params)
    
    import requests
    
    url = f'{api_url}?{params}'
    r = requests.get(url)
    data = r.json()
    return data

def init_db() -> pymysql.connections.Connection:
    # Open database connection
    # Connect to the database
    db = pymysql.connect(
        host='alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com',
         user='admin',
         password='adminadmin',
         cursorclass=pymysql.cursors.DictCursor
    )
    return db

def insert_data(df, db_name, table_name, db):
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    
    field_len = len(df.columns.values)
    cols = ','.join(list(df.columns.values))

    for i, row in df.iterrows():
        print(i)
        data_row = ','.join([row[list(df.columns.values)[field_i]] for field_i in range(field_len)])

        sql = f"""
            INSERT IGNORE INTO {db_name}.{table_name} ({cols})
            VALUES ({data_row});
        """
        print(sql.format(db_name, table_name,cols,data_row))

        cursor.execute(sql)
        
db = init_db()

In [ ]:
import pandas as pd

## daily ticker/currency (/api/dailyeod)
params_dict = {
    'function': 'FX_DAILY',
    'from_symbol': 'EUR',
    'to_symbol': 'USD',
    'apikey': 'demo'
}
data = get_alphavantage(params_dict)
# data

# df = pd.read_json(fname).reset_index()
df = pd.DataFrame(data).reset_index()

df = pd.concat([
    df[df.columns.tolist()[-1]].dropna().apply(pd.Series),
    df.drop(df.columns.tolist()[-1], axis=1)
], axis=1)

df = df[
    df[df.columns.tolist()[-1]].isnull() # take out last column with null value (Meta Data) - logic will change if json structure changes
].drop(
    df.columns.tolist()[-1], axis=1 # drop empty column (Meta Data)
)
df.rename(columns={'index': 'date'}, inplace=True)
df.columns = [x.replace('.','_').replace(' ','').lower() for x in df.columns]
df['date'] = df['date'].apply(lambda x: f"'{x}'")

df = df.reset_index(drop=True)
# df

table_name = '_'.join(list(params_dict.values())[:-1]).lower()
print(table_name)

insert_data(df, db_name, table_name, db)
db.commit()

In [ ]:
import pandas as pd

## monthly CPI (/api/monthlycpi)
params_dict = {
    'function': 'CPI',
    'interval': 'monthly',
    'apikey': 'demo'
}
data = get_alphavantage(params_dict)
# data

# df = pd.read_json(fname)
df = pd.DataFrame(data)

df = pd.concat([
    df[df.columns.tolist()[-1]].apply(pd.Series),
    df.drop(df.columns.tolist()[-1], axis=1)
], axis=1)
df = df[['date','value']]
df['date'] = df['date'].apply(lambda x: f"'{x}'")

df = df.reset_index(drop=True)
# df

table_name = '_'.join(list(params_dict.values())[:-1]).lower()
print(table_name)

insert_data(df, db_name, table_name, db)
db.commit()

## CLEAN DATA

## QUERY

In [10]:
import pandas as pd

query = f"SELECT * FROM alphavantage.fx_daily_eur_usd;"
df = pd.read_sql(query, db)
df

,date,1_open,2_high,3_low,4_close


In [6]:
db.close()

In [3]:
import pandas as pd

query = f"SELECT * FROM alphavantage_uat.cpi_monthly;"
df = pd.read_sql(query, db)
df

,date,value
0,1913-01-01,9.800
1,1913-02-01,9.800
2,1913-03-01,9.800
3,1913-04-01,9.800
4,1913-05-01,9.700
...,...,...
1300,2021-05-01,269.195
1301,2021-06-01,271.696
1302,2021-07-01,273.003
1303,2021-08-01,273.567


In [ ]:
db.close()

## CREATE TABLE - Adhoc

In [9]:
cursor = db.cursor()
    
# Drop table if it already exist using execute() method.
cursor.execute(f"DROP TABLE IF EXISTS alphavantage.fx_daily_eur_usd;")

# Create table as per requirement
sql = f"""
    CREATE TABLE IF NOT EXISTS alphavantage.fx_daily_eur_usd (
        date DATE PRIMARY KEY,
        1_open FLOAT,
        2_high FLOAT,
        3_low FLOAT,
        4_close FLOAT
    );
"""

cursor.execute(sql)

0

In [ ]:
cursor = db.cursor()

# Drop table if it already exist using execute() method.
cursor.execute(f"DROP TABLE IF EXISTS alphavantage.cpi_monthly;")

# Create table as per requirement
sql = f"""
    CREATE TABLE IF NOT EXISTS alphavantage.cpi_monthly (
        date DATE PRIMARY KEY,
        value FLOAT
    );
"""

cursor.execute(sql)

In [ ]:
list(i.values())[0]

In [ ]:
cursor = db.cursor()
cursor.execute("SHOW DATABASES;")

vals = list()
for i in cursor:
    vals.append(list(i.values())[0])
vals

In [ ]:
cursor = db.cursor()
cursor.execute("SHOW TABLES FROM alphavantage_uat;")

vals = list()
for i in cursor:
    print(i)
#     vals.append(list(i.values())[0])
vals

In [ ]:
cursor = db.cursor()
cursor.execute(f"DESCRIBE alphavantage_uat.cpi_monthly;")

vals = list()
for i in cursor:
    vals.append(list(i.values())[0])
vals

In [ ]:
import yaml

with open('/home/ec2-user/data/config.yml', "r") as f:
    data = yaml.safe_load(f)
data['DATABASE']['HOST']

In [ ]:
import os

# Set environment variables
os.environ['MYSQL_HOST'] = 'alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com'
os.environ['MYSQL_USER'] = 'admin'
os.environ['MYSQL_PASSWORD'] = 'adminadmin'

In [ ]:
import os

data = {
    'host':os.environ.get('MYSQL_HOST'),
    'user':os.environ.get('MYSQL_USER'),
    'password':os.environ.get('MYSQL_PASSWORD')
}
data

## MONTHLY

In [ ]:
# import requests

# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={key}'
# r = requests.get(url)
# data = r.json()

# print(data)

## CONNECT TO MySQL RDS

In [ ]:
# import pymysql

In [ ]:
# # Connect to the database
# connection = pymysql.connect(host='alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com',
#                              user='admin',
#                              password='adminadmin')
# #                              database='alphavantage',
# #                              cursorclass=pymysql.cursors.DictCursor)
# connection

In [ ]:
# cursor = connection.cursor()
# cursor

In [ ]:
# # execute SQL query using execute() method.
# cursor.execute("SELECT VERSION();")

# # Fetch a single row using fetchone() method.
# ver = cursor.fetchone()
# print ("Database version : %s " % ver)

## CREATE DATABASE

In [ ]:
# connection.cursor().execute('CREATE DATABASE alphavantage_uat;')

## SHOW DATABASES

In [ ]:
# cursor.execute("SHOW DATABASES;")

# for databases in cursor:
#     print(databases[0])

## THIRD PARTY API

In [ ]:
# from alpha_vantage.timeseries import TimeSeries
# import matplotlib.pyplot as plt

# ts = TimeSeries(key='GWXLIMM8PJ1W2F2C', output_format='json')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# print(data)
# print(meta_data)
# # data['close'].plot()
# # plt.title('Intraday Times Series for the MSFT stock (1 min)')
# # plt.show()